## 복수선택형 문제

In [1]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()
system_prompt = """당신은 프롬프트 엔지니어입니다. 아래 예시처럼 '새로운 질문'을 만드는 프롬프트를 짜려고 합니다. 

1. question과 original_question을 비교하고, 차이를 확인하세요. 
2. 옳은 것을 고르는 문제인지, 옳지 않은 것을 고르는 문제인지는 question에 따릅니다. 
3. question이 '어떤 주제에 대해' 옳은 것/옳지 않은 것을 고르는 것인지 명확하지 않다면 original_question의 질문과 questions의 '옳은 것/옳지 않은 것'을 조합해 질문을 만드세요. 
4. question에 주제가 있고, 옳은 것/옳지 않은 것을 고르는 것이 명확하다면 question을 그대로 사용하세요.

예시
'question': '옳은 것을 모두 고르시오.'
'original_question': '고객과 생산적인 대화를 위한 방법으로 다음 설명 중 가장 적절하지 않은 것은?'
-> 새로운 질문: '고객과 생산적인 대화를 위한 방법으로 다음 설명 중 옳은 것을 모두 고르시오.'
"""

# target_answer_count = 3
# options_ct = 5
# question = {}
# question['answer'] = "②"


user_prompt = f""""""
response = llm.query_openrouter(system_prompt, user_prompt)

In [2]:
print(response)

아래 프롬프트를 그대로 사용하세요.

작업:
- 주어진 question과 original_question을 비교해 ‘새로운 질문’ 한 줄을 생성한다.

입력:
- question: {여기에 입력}
- original_question: {여기에 입력}

규칙:
1) question과 original_question을 비교해
   - 주제(무엇에 대한 질문인지)
   - 판단 방향(옳은 것 vs. 옳지 않은 것, 적절한 vs. 부적절한 등)을 파악한다.
2) 최종 판단 방향은 question을 따른다. question에 방향이 없으면 original_question을 따른다.
3) question에 주제가 없거나 불명확하면, original_question에서 주제만 추출해 question의 형식/방향과 결합한다.
   - 주제 추출 시 ‘다음 중/설명 중/가장/옳은/옳지 않은/적절(하지) 않은/맞는/틀린/해당하지 않는’ 등 판단 표현은 제거하고, 대상/맥락만 남긴다.
4) question에 주제와 판단 방향이 모두 명확하면 question을 그대로 사용한다.
5) 어투와 종결(‘…고르시오.’, ‘…것은?’ 등)은 question의 형식을 최대한 유지한다.
6) 의미 왜곡 없이 자연스러운 한국어로 다듬되, 불필요한 수식어는 추가하지 않는다.
7) 출력 형식은 한 줄로만: 새로운 질문: {문장}

예시 입력:
- question: 옳은 것을 모두 고르시오.
- original_question: 고객과 생산적인 대화를 위한 방법으로 다음 설명 중 가장 적절하지 않은 것은?

예시 출력:
새로운 질문: 고객과 생산적인 대화를 위한 방법으로 다음 설명 중 옳은 것을 모두 고르시오.


In [ ]:
system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

검증
- 목표 정답 수(= '옳지 않은 것' 개수, {target_answer_count})가 2 이상 {options_ct} 이하인지 확인합니다. 범위를 벗어나면 오류를 보고합니다.
- 선택지 수({options_ct})와 순서는 반드시 유지합니다.

변형 규칙
- 문제 지문은 “옳지 않은 것을 모두 고르시오”로 바꿉니다. 그 외 본문, 수식(LaTeX), 표, 선택지 문구(변형 대상 제외)는 원형을 최대한 보존합니다.
- 목표 오답 수 = '옳은 것' 개수 = {options_ct - target_answer_count}.
  - 목표 오답 수 = 0: 원래 오답({question.get('answer', '')})을 최소 수정으로 ‘옳지 않음’으로 뒤집습니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 모든 선택지가 옳지 않음입니다.
  - 목표 오답 수 = 1: 원래 오답을 그대로 유지합니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 원래 오답인 '옳은 것' 1개만 남습니다(단일선택형 재검증).
  - 목표 오답 수 ≥ 2: 원래 오답을 그대로 유지하고, 추가로 (목표 오답 수 - 1)개 만큼 원래 옳지 않았던 선택지를 골라 최소 수정으로 ‘옳음’으로 만듭니다.
- 최소 수정 원칙
  - 허용되는 변경 예: 수치/단위/부등호/최대↔최소/있다↔없다/반드시↔경우도 있다/조건의 범위·한정어 조정 등.
  - 제공된 해설의 논리 범위를 벗어나는 임의 창작 금지. 외부 사실 의존 금지.
  - 변형 대상이 아닌 선택지의 문구는 절대 수정하지 않습니다.
- 선택지 선정 가이드
  - 변형이 필요한 경우, 가장 적은 토큰 변경으로 참⇄거짓을 뒤집기 쉬운 선택지부터 우선 선택합니다.
  - 의미 일관성 유지: 변형으로 인해 다른 선택지와 모순되거나 문제 전체의 전제가 깨지지 않도록 합니다.

정답과 설명
- 정답(answer)은 변형 후 ‘옳지 않은’ 선택지의 번호 목록입니다. 번호는 "①","②",... 형식의 문자열로, 오름차순으로 정렬합니다.
- explanation에는 모든 선택지를 순회하며 다음 형식으로 간단·명확히 기술합니다.
  - “① 옳다(원래 오답): 근거 …”
  - “③ 옳지 않다: 근거 …”
  - “⑤ 옳다(변형): 변경 ‘높다→낮다’, 근거 …”
- 변형된 선택지는 반드시 어떤 단어/수치/기호를 어떻게 바꿨는지 구체적으로 한 토큰 수준으로 표기합니다.

출력 형식(JSON)
{{
  "question_id": "문제번호",
  "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
  "options": ["① 선택지1", "② 선택지2", ...],  
  "answer": ["정답번호1", "정답번호2", ...], 
  "explanation": "① 옳다(원래 오답): … / ③ 옳지 않다: … / ⑤ 옳다(변형): 변경 ‘…→…’, 근거 …"
}}

비고
- 목표 오답 수 계산식으로 일관되게 제어합니다.
  - 추가로 만들어야 할 오답 수 = {max(0, options_ct - target_answer_count - 1)}.
  - 단, {options_ct - target_answer_count} = 0이면 원래 오답을 ‘옳지 않음’으로 뒤집고 추가 오답은 0입니다.
- 선택지 레이블(①, ②, …)과 원문 순서를 유지합니다.
- 최종적으로 옳지 않음 개수 = {target_answer_count}, 오답(옳음) 개수 = {options_ct - target_answer_count}임을 자기점검합니다.
"""

### 피드백 반영

In [ ]:
atc[132]

In [ ]:
user_prompt = f"""
=========== 문제 ==========
"""
p = atc[132]
question_id = p['file_id'] + '_' + p['tag']
question = p['question']
options = p['options']
answer = p['answer']
explanation = p['explanation']

user_prompt += f"""문제번호: {question_id}
문제: {question}
선택지: {options}
답: {answer}
해설: {explanation}
"""

In [ ]:
# 입력
# - options_ct: 원문 선택지 수
# - i: 변형 후 정답(‘옳은 것’)의 개수
# - given_wrong_idx: 원래 ‘옳지 않은’ 선택지 번호(필수)

question = atc[132]
options_ct = len(options)
target_answer_count = 2
given_wrong_idx = answer

system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

검증
- 목표 정답 수(= '옳지 않은 것' 개수, {target_answer_count})가 2 이상 {options_ct} 이하인지 확인합니다. 범위를 벗어나면 오류를 보고합니다.
- 선택지 수({options_ct})와 순서는 반드시 유지합니다.

변형 규칙
- 문제 지문은 “옳지 않은 것을 모두 고르시오”로 바꿉니다. 그 외 본문, 수식(LaTeX), 표, 선택지 문구(변형 대상 제외)는 원형을 최대한 보존합니다.
- 목표 오답 수 = '옳은 것' 개수 = {options_ct - target_answer_count}.
  - 목표 오답 수 = 0: 원래 오답({question.get('answer', '')})을 최소 수정으로 ‘옳지 않음’으로 뒤집습니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 모든 선택지가 옳지 않음입니다.
  - 목표 오답 수 = 1: 원래 오답을 그대로 유지합니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 원래 오답인 '옳은 것' 1개만 남습니다(단일선택형 재검증).
  - 목표 오답 수 ≥ 2: 원래 오답을 그대로 유지하고, 추가로 (목표 오답 수 - 1)개 만큼 원래 옳지 않았던 선택지를 골라 최소 수정으로 ‘옳음’으로 만듭니다.
- 최소 수정 원칙
  - 허용되는 변경 예: 수치/단위/부등호/최대↔최소/있다↔없다/반드시↔경우도 있다/조건의 범위·한정어 조정 등.
  - 제공된 해설의 논리 범위를 벗어나는 임의 창작 금지. 외부 사실 의존 금지.
  - 변형 대상이 아닌 선택지의 문구는 절대 수정하지 않습니다.
- 선택지 선정 가이드
  - 변형이 필요한 경우, 가장 적은 토큰 변경으로 참⇄거짓을 뒤집기 쉬운 선택지부터 우선 선택합니다.
  - 의미 일관성 유지: 변형으로 인해 다른 선택지와 모순되거나 문제 전체의 전제가 깨지지 않도록 합니다.

정답과 설명
- 정답(answer)은 변형 후 ‘옳지 않은’ 선택지의 번호 목록입니다. 번호는 "①","②",... 형식의 문자열로, 오름차순으로 정렬합니다.
- explanation에는 모든 선택지를 순회하며 다음 형식으로 간단·명확히 기술합니다.
  - “① 옳다(원래 오답): 근거 …”
  - “③ 옳지 않다: 근거 …”
  - “⑤ 옳다(변형): 변경 ‘높다→낮다’, 근거 …”
- 변형된 선택지는 반드시 어떤 단어/수치/기호를 어떻게 바꿨는지 구체적으로 한 토큰 수준으로 표기합니다.

출력 형식(JSON)
{{
  "question_id": "문제번호",
  "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
  "options": ["① 선택지1", "② 선택지2", ...],  
  "answer": ["정답번호1", "정답번호2", ...], 
  "explanation": "① 옳다(원래 오답): … / ③ 옳지 않다: … / ⑤ 옳다(변형): 변경 ‘…→…’, 근거 …"
}}

비고
- 목표 오답 수 계산식으로 일관되게 제어합니다.
  - 추가로 만들어야 할 오답 수 = {max(0, options_ct - target_answer_count - 1)}.
  - 단, {options_ct - target_answer_count} = 0이면 원래 오답을 ‘옳지 않음’으로 뒤집고 추가 오답은 0입니다.
- 선택지 레이블(①, ②, …)과 원문 순서를 유지합니다.
- 최종적으로 옳지 않음 개수 = {target_answer_count}, 오답(옳음) 개수 = {options_ct - target_answer_count}임을 자기점검합니다.
"""

In [ ]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()
response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'openai/o3')
print(response)

In [ ]:
question

## 서술형 프롬프트 짜기

In [3]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()

In [1]:
system_prompt = """당신은 프롬프트 엔지니어입니다. 아래와 같은 원리를 담아 주어진 객관식 문제를 서술형 문제로 변형하는 프롬프트를 만들려고 합니다.

1. 기존 객관식 문제의 질문에서 주제 추출 (ex. 코넥스상장법인의 기업설명회에 대한 설명 중 잘못된 것은? -> 코넥스상장법인의 기업설명회)
2. 추출한 내용을 바탕으로 서술형 문제로 변형 (ex. 코넥스상장법인의 기업설명회에 대해 서술하시오)
3. 1개의 옳지 않은 선지와 그 해설, 3~4개의 옳은 선지와 그 해설에서 각 선지별로 키워드 추출 (ex. ① 코넥스상장법인은 분기마다 기업설명회를 개최해야 한다. -> 기업설명회 개최 주기)
4. 키워드를 포함한 서술형 문제 완성 (ex. 다음 키워드를 활용하여 코넥스상장법인의 기업설명회에 관해 서술하시오.\n 키워드: 기업설명회 개최 주기, 상장폐지사유, 공시 의무, ...)

단, 키워드는 10개 이하여야 합니다. 키워드는 너무 정답이 드러나지 않도록 주의합니다.
선지와 해설만을 이용해서 서술형 문제로 변형할 수 있는 프롬프트를 작성해주세요."""

q = {'file_id': 'SS0206',
 'title': '2025 해커스 증권투자권유대행인 한권합격 핵심 개념 + 적중문제',
 'chapter': '제1과목 증권분석 및 증권시장',
 'tag': 'q_0211_0003',
 'domain': '자산운용',
 'subdomain': '투자분석 및 포트폴리오이론',
 'classification_reason': '코넥스 상장법인 기업설명회(개최 주기, 공시 의무, 신고, 상장폐지)로 증권 시장 상장 규정과 공시 중심',
 'is_calculation': False,
 'question': '코넥스상장법인의 기업설명회에 대한 설명 중 잘못된 것은?',
 'options': ['① 코넥스상장법인은 분기마다 기업설명회를 개최해야 한다.',
  '② 코넥스상장법인이 기업설명회를 개최하는 경우 일시, 장소, 설명회 내용 등을 공시해야 할 의무가 있다.',
  '③ 코넥스상장법인은 기업설명회 개최 결과를 개최일 익일까지 거래소에 신고해야 한다.',
  '④ 코넥스상장법인이 2반기 동안 연속하여 기업설명회를 개최하지 않거나, 3년 동안 4회 이상 개최하지 않은 경우 상장폐지사유에 해당한다.'],
 'answer': '①',
 'explanation': '코넥스상장법인은 반기마다 기업설명회를 개최해야 한다.',
 'answer_type': 'wrong'}

user_prompt = f"""
==== 문제 예시 ====
질문: {q['question']}
선택지: {q['options']}
정답: {q['answer']}
해설: {q['explanation']}
"""

In [4]:
response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'openai/gpt-5')
print(response)

다음 지시문을 따르세요. 당신의 역할은 주어진 객관식 항목(선지)과 해설만을 근거로 문제의 핵심 주제를 추론하고, 이를 서술형 문제로 변환하는 것입니다.

입력으로 제공되는 것
- 선택지: 문자열 배열
- 정답: 정답 표기(예: '①' 또는 인덱스)
- 해설: 정답에 관한 설명 또는 정오 수정 설명

필수 규칙
1) 자료 제한: 선택지와 해설만 사용하세요. 원 질문 문구가 있더라도 주제 추론에 활용하지 마세요.
2) 주제 추출: 선택지와 해설에서 반복적으로 등장하는 핵심 명사구를 찾고, 가장 포괄적·간결한 1개의 주제를 도출하세요.
3) 서술형 변환: “[주제]에 관해 서술하시오.” 형태로 만들되, 앞에 “다음 키워드를 활용하여”를 붙여 한 문장으로 제시하세요.
4) 선지 분류: 해설을 근거로 틀린 선지 1개와 옳은 선지 3~4개를 식별하세요. 옳은 선지가 3개 미만이면 가능한 만큼만 사용하세요.
   - 정답 표기가 있는 경우, 해설과의 정합성을 우선으로 판단하여 틀린 선지를 확정하세요.
5) 키워드 추출: 틀린 선지 1개와 옳은 선지 3~4개 각각에서 핵심 개념을 1개씩 뽑아 키워드를 구성하세요. 총 5~10개 이내로 하며, 다음 원칙을 지키세요.
   - 정답이 바로 드러나지 않도록 수치·주기·임계치·예외 조건·직접적인 정오 표현을 추상화하십시오.
   - 예: “반기마다 개최” → “개최 주기”, “익일 신고” → “보고 시한”, “상장폐지사유” → “제재 요건”
   - 중복 키워드는 제거하고, 명사구 중심으로 간결히 표현하세요.
6) 출력은 정확히 두 줄만 생성하세요.
   - 1행: 서술형 문제: 다음 키워드를 활용하여 [주제]에 관해 서술하시오.
   - 2행: 키워드: 키워드1, 키워드2, 키워드3, …

검증 체크리스트
- 키워드는 10개 이내인가?
- 키워드에 구체 수치·주기·정답을 직설적으로 드러내는 표현이 없는가?
- 주제 문구가 명확하고 간결한가?

예시 입력 형식
선택지: ['① ...', '② ...', '③ ...', '④ ...